# 7. Amplification of extracellular signals

<hr>


**Design principles**

- Bifunctional kinases that paradoxically phosphorylate _and_ dephosphorylate their targets enable linear amplification that is robust to total protein copy numbers.
- Biphasic control can be accomplished with paradoxical regulation.
- Kinase cascades can yield both ultrasensitive and graded responses using the same architecture.

**Concepts**

- Paradoxical regulation

**Techniques**

- Applying conservation laws to simplify circuit analysis.

<hr>

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
# ------------------------------

import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

## Amplifiers: the middle management of biological circuitry

In electronic circuits, among the first systems one learns about are amplifiers. They come in a huge variety of types and are required for a vast array of circuit capabilities. Biological circuits are similarly overflowing with amplifiers, which play a variety of essential roles in circuits. The cell 

First and most obviously, amplifiers can modulate the amplitude of a signal. This is important if an intrinsically weak input signal needs to be magnified to control a cellular response. Second, amplifiers can reshape signals, allowing a system to respond to signals that exceed a threshold. A third, less obvious aspect of amplifiers is their role in conveying signals from one molecular species (e.g. protein) or biochemical form to another (e.g. from protein abundance to protein phosphorylation).

<!-- JB: In the first point, an amplifier need not be perfectly linear to increase the amplitude. Maybe introduce perfect linearity later?
-->

Amplifiers are like middle managers: they do not generate new information nor do they use information. Instead, they take results from one group, extract key features, manipulate them, and then pass them along to others in a new form, hopefully adding some value along the way. 

In this chapter, we will analyze multi-component amplifiers based on protein kinases. While  gene regulation also functions as a kind of amplifier, with transcription factors serving as inputs to control the abundances of other genes, phosphorylation amplifiers can operate faster and more directly. We will focus on two of the most important classes of kinasee amplifiers: two-component systems in prokaryotes and kinase cascades in eukaryotes. We will learn how amplifiers can operate in a manner that is robust to their own component concentrations and how they can allow tuning of two key properties: gain and sensitivity.  

## Amplifiers are characterized by gain and sensitivity

Consider the task of designing a biomolecular sensor module that encodes the concentration of an input signal in the concentration of a target protein. Cells have to do this all the time, in order to transmit information about the external environment from sensory components at the periphery of the cell to regulatory components in the interior of the cell that regulate the cellular response to environmental conditions. As they convert information from the input to the effector molecule, in a process called **signal transduction**, they also amplify it, making the effector response larger than the input. The degree to which an output signal is increased relative to its input is called **gain.** Gain is easiest to define for a linear input-output relation, where it is independent of input level. In non-linear input-output systems, gain varies depending on the input level. More importantly, non-linear systems offer different levels of **sensitivity**, defined as $\mathrm{log}(\mathrm{output}/\mathrm{input})$. Sensitivity, like gain, is a local property. For a Hill function, $f(x) = {x^n}/\left({x^n + k^n }\right) $, it is maximal at the input level that produces a half-maximal response, $x=k$, and $0$ when $x \ll k$ or $x \gg k$.

We have already seen how the sensitivity of a regulation function can critically influence the functional behavior of the circuit in which it appears. For example, in Chapter 3 we saw that ultrasensitive regulation ($n > 1$) was required for bistability in both the positive autoregulatory feedback loop and mutual repression toggle switch. On the other hand, we saw in Chapter 5 that incoherent feed-forward loops enable dosage compensation when regulation is linear ($n=1$). Thus, depending on the circuit function, different levels of sensitivity can be necessary.

Different kingdoms provide different amplification circuits. Bacteria rely heavily on a diverse set of signaling pathways called two-component systems. Mammalian cells use  distinct families of signal transduction systems, often based on cascades of kinases that regulate one another in linear cascades. In this chapter we will consider both types of signaling systems.

### Tradeoffs between sensitivity and fidelity

If the relationship between the output concentration and the input concentration is ultrasensitive, the module can reliably encode the *qualitative* information of whether the input signal is significantly above or below a particular threshold. However, it performs poorly in encoding the *quantitative* information of the input signal's exact concentration. With an ultrasensitive response, most input concentrations either fail to activate or saturate response, causing most input concentrations to map to similar output concentrations. Further, even within the sensitive part of the response curve, slight fluctuations in the input concentration can lead to large variations in the output. As a result, ultrasensitive systems do a poor job at allowing the cell to confidently infer the  input concentrations. (The chemotaxis circuit we encountered in the previous chapter gets around this problem by continually adapting to ambient input signals, effectively keeping itself responsive to small relative changes in input, but throwing away information about absolute signal levels.) 

<!--
JB: We need to programmatically generate the plot below.
-->

<div style="width: 650px; margin: auto;">
    
![nonlinear and linear response function](figs/NonLinear-LinearResponseSchematic.png)

</div>

A better choice for this task would be to use a linear (non-ultrasensitive) response function, in which the output concentration is simply proportional to the input concentration over the relevant range of input signals with the gain $g$, determining the degree of amplification. $g > 1$ indicates amplification, while $g < 1$ indicates de-amplification.

\begin{align}
\mathrm{output} = g \times  \mathrm{input}
\end{align}

With such a system, fluctuations in the input propagate to the output but are not disproportionately amplified. 

Two conditions are necessary to make such a "perfect" linear amplifier. First, **linearity**: the linear response regime should cover the range of input concentrations the system needs to reliably encode. Second, **robustness**: since all protein levels vary between cells and over time,  the entire input-output response curve should ideally remain the same regardless of the levels of the systems own components. 

What circuit design could provide such a linear input-output response and ensure it is robust to variation in its own components? 

## Two-component signaling systems provide tunable linear amplification.

**Two-component signaling systems** are ubiquitous in both prokaryotes and eukaryotes. They modularly encode the ability to sense a signal, via a sensing receptor, and respond to it, via a cognate response regulator, often a transcription factor, whose activity is modulated by the receptor component. The "two components" in the name refer to a histidine kinase that can autophosphorylate itself on a histidine residue and then transfer these phosphate groups to an aspartate residue on a second protein termed the response regulator. Once phosphorylated, this protein can activate target genes or other processes. Bacteria often have many distinct two component systems. In fact, the chemotaxis system we discussed in the previous chapter is a special example of a two component signaling system. 

(Many two-component systems indeed have two protein components, as the name implies, while others actually have more, making the term, in some cases, a misnomer. For example, Bacillus subtilis cells use a four-component "phosphorelay" in which histidine kinases of the general type considered here respond to environmental insults by autophosphorylating and transferring their phosphate to a second protein, from which it is then transfered to a third, and finally a fourth protein, which is thereby activated. This terminal protein, called Spo0A, triggers sporulation, the transformation of the living cell into a dormant spore. This phosphorelay is used to sense dire environmental conditions and respond by triggering sporulation. An interesting, and as far as we know not fully answered, design question is under what circumstances a system should use such a 4-component phosphorelay rather than the seemingly simpler 2-component architecture described below.)  

### Two-component system input-output relationships generally depend on the concentrations of circuit components

Before we get to the two component system architecture, we will first observe how hard the problem  of robust, linear amplification is, by considering the simplest signaling system we can imagine. Here, an input signaling molecule $S$ allows the receptor kinase $X$ to phosphorylate a signaling moleucle $Y$, as shown below.

<div style="margin: auto; width: 200px;">

![nonparadoxical signaling](figs/Two-component-system_minimal.png)
                                                            
</div> 

Using the analysis techniques for Michaelis-Menten kinetics that we introduced in the previous chapter, we can determine the system's response behavior.
If we assume the enzymes operate at saturation, we have

\begin{align}
\frac{\mathrm{d}y_p}{\mathrm{d}t} = v_1(s)\,x \, y - v_2\,z \, y_p.
\end{align}

We also have conservation of mass. Defining $y_0$ to be the total concentration of Y, we have $y_0 = y + y_p$. Using this expression, we have

\begin{align}
\frac{\mathrm{d}y_p}{\mathrm{d}t} = v_1(s)\,x \, (y_0-y_p) - v_2\,z \, y_p.
\end{align}

We can solve for the steady state concentration of phosphorylated Y, which will affect the expression of the target gene, by setting $\mathrm{d}y_p/\mathrm{d}t = 0$ and solving. The result is

\begin{align}
y_p(s) = \frac{v_1(s)\,x}{v_1(s)\,x + v_2\,z}\,y_0.
\end{align}

Looking at our expression for $y_p(s)$, we see that it takes a similar functional form to a Hill activation function of $v_1(s)$ with $n=1$, which can indeed be approximated by a linear function in the regime $v_1(s)/v_2 \ll z/x$. 

Notice, however, that the expression for this threshold value, $v_2 z/x$, includes terms for the concentrations of Z and X. Normally in a Hill function, this threshold is given by a biochemical parameter, $k$, which represents the binding affinity of the regulator to its binding site. Such a parameter is not expected to change significantly within a given cellular environment. By contrast, in our $y_p(s)$ expression, variations in the concentrations of Z and X will dynamically alter the location of the threshold for linearity, changing the overall shape of the response function itself! Since the target gene has no way to directly sense the concentrations of X and Z, this will significantly compromise the ability of the output to faithfully encode the concentration of the input. You can see how the response function changes as X and Z change in the interactive plot below.

In [2]:
# Initial parameters
x = 3
z = 0.5
v2 = 1

# x-y data for plotting
v1s = np.linspace(0, 3, 200)
yp = v1s * x / (v1s * x + v2 * z)

# Place the data in a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(v1s=v1s, yp=yp))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=150,
    frame_width=300,
    x_axis_label="v₁(s)∕v₂",
    y_axis_label="steady-state yₚ∕y₀",
    title="Simple Two-Component System",
    x_range=[0, 3],
    y_range=[-0.025, 1.025],
    tools="save",
)

p.line(source=cds, x="v1s", y="yp", line_width=2)

x_slider = bokeh.models.Slider(
    title="x/y₀", start=0.01, end=5, step=0.01, value=x, width=150
)
z_slider = bokeh.models.Slider(
    title="z/y₀", start=0.01, end=5, step=0.01, value=z, width=150
)


js_code = """
let yp = cds.data['yp'];
let v1s = cds.data['v1s'];
let x = x_slider.value;
let z = z_slider.value;

for (let i = 0; i < yp.length; i++) {
  yp[i] = v1s[i] * x / (v1s[i] * x + z);
}

cds.change.emit();
"""

callback = bokeh.models.CustomJS(
    args=dict(
        cds=cds,
        x_slider=x_slider,
        z_slider=z_slider,
        yaxis=p.yaxis,
    ),
    code=js_code,
)
x_slider.js_on_change("value", callback)
z_slider.js_on_change("value", callback)

layout = bokeh.layouts.row(
    p,
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(bokeh.models.Spacer(height=15), x_slider, z_slider),
)

bokeh.io.show(layout)

To overcome this obstacle, and improve the fidelity of $y_p$'s encoding of $s$, we need $y_p$ to somehow depend *only* on the concentation of the signaling molecule, $s$, independent of the concentrations of X, Y, and Z. But how...?

### Bifunctional kinases make two-component signaling systems robust to variation in their own components

The solution to this conundrum comes from a perplexing feature of many two-component systems: The  same histidine kinase protein responsible for transferring phosphates to the response regulator also has a distinct, and nearly opposite, activity as a phosphatase. This kinase is therefore said to be "bifunctional." In that role, it causes the response regulator to _lose_ its phosphate. Perversely, even as it giveth of the phosphate, it also taketh away. This is an example of a more general concept called **paradoxical regulation**, in which the same component can have two opposite effects on its target.

To see how this particular type of paradoxical regulation impacts the input-output behavior of the system, we will write down a modified model, in which the receptor X is a bifunctional kinase. In its phosphorylated form, it phosphorylates Y via phosphotransfer with rate $v_3$. This is its activating regulation. In its dephosphorylated state, it serves to dephosphorylate Y with rate $v_2$, giving it its paradoxical repressive regulation. So, X paradoxically has both kinase (phosphotransfer) and phosphatase (dephosphorylation) activity on the same substrate protein, Y. It also gets phosphorylated in contact with the signaling molecules S through consumption of an ATP. This circuit consumes energy by phosphorylating and dephosphorylating the same substrate, an example of a **futile cycle**.

<div style="margin: auto; width: 200px;">

![paradoxical signaling](figs/bifunctional_two_component.png)
                                                            
</div> 

We could write out the full dynamics of this system and solve for the steady state. However, in this case there is a shortcut, introduced by [Shinar et al.](https://doi.org/10.1073/pnas.0706792104) One can think of the multi-reaction system as a "black box." ATP goes in, to enable kinase autophosphorylation, and ADP and inorganic phosphate come out, through the phosphatase reaction. 

<div style="margin: auto; width: 250px;">

![ATP black box](figs/black_box.png)
                                                            
</div> 



Conservation of mass dictates that the total flux of this phosphate into the system must match the total flux of Pi out of the system. The influx is $v_1(s)\,x$ and the outflux is $v_2\,x\,y_p$. Setting these fluxes to be equal gives

\begin{align}
v_1(s)\,x = v_2\,x\,y_p,
\end{align}

so that

\begin{align}
y_p = \frac{v_1(s)}{v_2}.
\end{align}

This is not quite complete, since we still need to respect conservation of mass of Y, so

\begin{align}
y_p = \begin{cases}
\frac{v_1(s)}{v_2} & y_0 \ge \frac{v_1(s)}{v_2},\\[1em]
y_0 & y_0 < \frac{v_1(s)}{v_2}.
\end{cases}
\end{align}

Evidently, $y_p$ depends only on $s$, and is _completely independent_ of the total amounts of the proteins X and Y (up to the ceiling imposed by $y_0$). Importantly, the activity, as quantified by the concentration of phosphorylated Y, is linear in the input $v_1(s)$, with a slope of $1/v_2$. Thus, this system, at least before saturating, is a **perfect linear amplifier** of the input signal. We demonstrate what this function looks like in the plot below.

In [3]:
# Parameters
y0 = 1
v1s_max = 1.5

# Calculate functions
v1s_1 = np.array([0, y0])
v1s_2 = np.array([y0, v1s_max])

yp_1 = v1s_1 / y0
yp_2 = np.ones(len(v1s_2))

v1s = np.concatenate((v1s_1, v1s_2))
yp = np.concatenate((yp_1, yp_2))

# Build plot
p = bokeh.plotting.figure(
    frame_height=150,
    frame_width=300,
    x_axis_label="v₁(s)∕v₂",
    y_axis_label="yₚ",
    x_range=[0, y0 * 1.5],
    tools="save"
)

p.xaxis.ticker = [0, 0.5, 1.0, 1.5]
p.xaxis.major_label_overrides = {0.5: 'y₀∕2', 1: 'y₀', 1.5: '3y₀∕2'}
p.yaxis.ticker = [0, 0.5, 1.0]
p.yaxis.major_label_overrides = {0.5: 'y₀∕2', 1: 'y₀'}


# Plot function
p.line(v1s, yp, line_width=2)

bokeh.io.show(p)

Key to the linear amplification of signal is the following.

1. The cell must produce enough Y to make sure it does not saturate for the anticipated range of input values. That is, $y_0$ must be at least $v_1(s_\mathrm{max})/v_2$, where $s_\mathrm{max}$ is the maximum expected concentration of the signaling molecule.
2. The gain of the amplifier (the slope of the linear response) is inversely proportional to the dephosphorylation rate $v_2$. Slow dephosphorylation gives a larger gain.
3. It is critical that only the dephosphorylated state of the bifunctional kinase X have phosphatase activity. (If the phosphorylated state could act as a phosphatase, the inorganic phosphate balance changes, and we lose the linear amplification feature.)
4. All enzymes are operating at saturation. We would lose perfect linear amplification due to nonlinearities if they were not.
5. We are neglecting other slow reactions. In particular, spontaneous dephosphorylation, independent of the histidine kinase, could reduce robustness to total protein levels.

<!--
<div class="alert alert-block alert-info">
Potential exercise: build a ODE system that models this system with the Michaelis Menten kinetics and put it into parameter regimes that break each of these assumptions. See how the resulting $y_p(s)$ behavior changes.
</div>


<div class="alert alert-block alert-info">
Potential exercise: Take the original non-paradoxical signaling system and include an explicit ATP to ADP step associated with $X$'s phosphorylation of $Y$, and an explicit formation of inorganic phosphate associated with $Z$'s dephosphorylation of $Y_p$. Then apply the same black box appraoch that was used to analyze the paradoxical system, and show that you obtain the same expression for $y_p(s)$ as we obtained using michaelis menten analysis.
</div>
-->

### An example paradoxical signaling system

The EnvZ-OmpR system in _E. coli_ is a classic two component system with a bifunctional kinase. EnvZ is a histidine kinase membrane protein that senses osmotic stress. It phosphorylates OmpR, which in turn regulates expression of porin genes. At high osmolarity, the cell up-regulates a large porin called OmpF, while at lower osmolarity, it predominantly expresses OmpC, which has a smaller pore. 

<div style="margin: auto; width: 250px;">

![EnvZ-OmpR](figs/EnvZ-OmpR.png)
                                                            
</div> 


Batchelor and Goulian ([PNAS, 2003](https://doi.org/10.1073/pnas.0234782100)) engineered _E. coli_ cells to express OmpR under the control of the _lac_ promoter so they could systematically vary its expression by titrating the lactose analog IPTG. They then monitored expression of fluorescent reporter proteins expressed from OmpR's target promoters. The compared target promoter activity at low osmolarity (in a minimal medium) and at high osmolarity (minimal medium with 15% sucrose). The result of their experiment is shown below (using data digitized from the paper).

In [4]:
data_low = np.array(
    [
        [-0.9731, 0.5020],
        [-0.4584, 0.9084],
        [0.0087, 0.9920],
        [0.2800, 0.8486],
        [0.5300, 0.7171],
        [0.9079, 0.7649],
        [1.0654, 0.9084],
        [1.0996, 0.9920],
        [1.1593, 1.1116],
        [1.3748, 2.2709],
    ]
)

data_high = np.array(
    [
        [-0.9787, 0.2777],
        [-0.6626, 7.6505],
        [-0.1894, 11.3891],
        [0.2365, 11.5047],
        [0.2833, 11.5021],
        [0.4325, 12.1910],
        [0.6796, 12.5956],
        [0.9735, 12.9975],
        [1.0717, 15.0837],
        [1.1288, 29.1641],
    ]
)

p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=450,
    x_axis_label="fold increase in OmpR",
    y_axis_label="target promoter activity (a.u.)",
    x_axis_type="log",
    y_axis_type="log",
)
p.circle(10 ** data_low[:, 0], data_low[:, 1], legend_label="low osmolarity")
p.circle(
    10 ** data_high[:, 0],
    data_high[:, 1],
    color="orange",
    legend_label="high osmolarity",
)
p.legend.location = "center"
bokeh.io.show(p)

Astoundingly, over at least an order of magnitude the target promoter activity is robust to OmpR concentration, sensitive only to the osmolarity of the surrounding environment. This suggests that the system indeed achieves robust regulation. 

<!--

<div class="alert alert-block alert-info">
MBE To do: Add discussion of linear responses from product inhibition: Springer & colleagues: https://www.cell.com/cell-systems/fulltext/S2405-4712(15)00114-3
    - Check whether model is robust to component concentrations.
</div>

-->

## Encoding ultrasensitivity: beyond cooperativity

Let's now return back to our discussion about ultrasensitivity, above. We have seen that bifunctional two-component systems can generate almost perfect linear amplifiers through their paradoxical architecture, and thus represent ideal systems for transmitting a signal with minimal distortion.

But it is also sometimes the case that cells want to respond in more of an all-or-none manner to an external stimulus. This is particularly true for yes/no decisions. Should the cell divide? Should it differentiate? Should it undergo cell death? All of these responses require a sharp, switch-like response to input signals. 

When we introduced ultrasensitivty in Chapters 2 and 3 via the Hill coefficient $n$, we pointed out that it is often achieved by cooperativity in regulatory genes. If the monomers of a transcription factor must combine to form a dimer before it can regulate its target genes, then we would expect to see a Hill coefficient up to $n \approx 2$. Similarly, transcription factors that are only active as tetramers could yield a Hill coefficient of up to $n \approx 4$. While this mechanism can generate ultrasensitivity, it also has some disadvantages.

For one, generating very ultrasensitive responses ($n > 10$, for example) using only multimerization would require very large complexes. This places an architectural constraint on the structure of the proteins themselves which may compromise their other functional properties. Secondly, cooperativity is a property that is often 'hard-wired' into the structure of the protein itself—it is difficult to dynamically tune whether a protein complex is active only as a tetramer, only as a dimer, or only as a monomer, for example. Cells may very well face situations where they want to respond ultrasensitively to a signal, such as a hormone, in one context or cell type, but sense that same signal more linearly in a different one. In such a case, a more flexible, and dynamically tunable system for controlling ultrasensitivity would be useful.

In this section, we will explore the kinase cascade circuit and show how the expression levels of its components can tune from graded to ultrasensitive responses.

## The MAP kinase cascade exhibits both ultrasensitive and graded responses

The MAP (Mitogen-Activated Protein) kinase cascade is a conserved signaling system found in all eukaryotes, including yeast, plants, and mammals. Its core structure consists of a cascade of three kinases, including the eponymous MAP kinase itself (MAPK), whose active, phosphorylated form directs a downstream response. MAPK is switched into this active form through phosphorylation by the MAPK Kinase (MAPKK). MAPKK, in turn, is also active only in a phosphorylated form, which it can enter via the action of the MAPKK Kinase (MAPKKK). And MAPKKK too has active and inactive forms. As the upstream kinase in the cascade, it is phosphorylated by a variety of signals and inputs, such as growth factor receptors. Interestingly, these kinases often need to be doubly phosphorylated on two sites in order to be fully active, as indicated below by the "PP" forms. 

<div style="width: 450px; margin: auto;">
    
![MAP kinase cascade schematic](figs/huang_MAPK_schematic.png)

</div>

*Image from Huang and Ferrell 1996, PNAS*

This core pathway is duplicated in some systems, typically embedded within larger circuits, and sometimes elaborated with additional regulatory inputs. Nevertheless, the core MAPK cascade architecture is strikingly conserved and provides critical functions across a huge range of different biological systems and contexts. 

Because of its central role in many critical cellular processes, the MAPK cascade has been well-studied by biologists for decades. This body of work has revealed that, in various organisms, the MAPK cascade can both exhibit an ultrasensitive response (see [Huang and Ferrel 1996, PNAS](https://doi.org/10.1073/pnas.93.19.10078)) and a graded response (see [Poritz et al 2007, Yeast](https://doi.org/10.1002/yea.777)), as well as many other interesting dynamical properties ([Bhalla et al 2002, Science](https://doi.org/10.1126/science.1068873)). In particular, James Ferrell and colleagues did pioneering early studies that sought to [quantitatively understand](http://dx.doi.org/10.1016/j.tibs.2014.10.002) the nature of amplification in kinase cascades.




At first glance, the structure of this system appears puzzling. Why are all these intermediate kinases necessary? Can the first kinase not just directly active the downstream response, and skip the middle men (MAPKKK and MAPKK)? Has the MAPK cascade indeed evolved to produce different behaivors in each organism and context? Or, could it be that a key conserved function of this cascade is precisely its functional plasticity, i.e. its ability to operate in various different modes in different cell contexts within each species. 

In order to address this question, one would ideally like to to take a MAPK cascade and systematically analyze its input-output response across a wide range of different regimes, varying the levels of its components. [O'Shaughnessy et al. (2011, Cell)](https://doi.org/10.1016/j.cell.2010.12.014) used a synthetic biology approach to do just this. They built a minimal mammalian MAPK cascade in yeast. This transplantation approach helped to insulate the core motif from the many complicating regulatory interactions it would otherwise have in its native mammalian context. Importantly, the authors designed their system so that they could quantitatively control the concentration of each of the system components. Using these "control knobs," they investigated the response profile of the cascade across these different regimes. They then compared the observed experimental behavior with expectations from a mathematical model, which we will now explore.

We will see that the extra steps in the cascade provide 'control knobs' that allow it to operate it as a tunable amplifier. 


### A mathematical model of the MAP kinase cascade

O'Shaughnessy et al. developed a mass action ODE model based on the following reaction scheme, where the environmental signal (S) propagates down through the sequential kinases until the final MAP kinase, ERK, reaches its active form (ERKpp).

<div style="width: 400px; margin: auto;">
    
![MAP kinase cascade reaction schematic](figs/MAPK_cascade.png)

</div>

We denote the three cascade kinases as $R$, $M$, and $E$, respectively, for Raf, MEK, and ERK. To represent the input, the authors assume that S binds directly to R to create an active Raf kinase, denoted $[R \cdot S]$. Each subsequent phosphorylation step is treated as a Michaelis-Menten reaction (see Chapter 6). For example, $M + [R \cdot S] \leftrightarrow [M \cdot R \cdot S] \rightarrow M_p + [R \cdot S]$, where $M_p$ denotes a singly phosphorylated state of MEK. $M_p$ can similarly be phosphorylated to produce the active, doubly phosphorylated state, $M_{pp}$.

For each step in the cascade, one needs to specify:
* Whether the multiple phosphorylation sites on the target protein are phosphorylated **processively**, where a single encounter with the kinase causes phosphorylation of all sites, or **distributively**, requiring independent kinase encounters for each site. Below, we will assume distributive phosphorylation.
* Whether distinct sites can be phosphorylated only in a sequential order, or in a random order. Below, we assume sequential phosphorylation. 
* How the phosphorylation status at each site combine to control the kinase activity of the protein. For example, whether they combine through AND, OR, or some other logic. Below, we assume AND logic.
* What the concentrations and $K_m$ values are for each reaction, and more generally whether the kinases are operating in saturated or unsaturated regimes.

Denoting concentrations using square brackets in order to make the notation for complexes clear, and introducing a set of numbered rate constants, $k_i$ and $k_{-i}$, for the various forward and reverse reactions, respectively, the authors obtained a formidable set of ODEs:

<!--
These equations needs to be reformatted.
-->

\begin{align}
&\frac{\mathrm{d}[R]}{\mathrm{d}t} = \beta_R - \gamma [R] - k_1 [R][S] + k_{-1} [R\cdot S] \\[1em]
&\frac{\mathrm{d}[R\cdot S]}{\mathrm{d}t} =  - \gamma [R \cdot S] + k_1 [R][S] - k_{-1} [R\cdot S] - k_2 [M][R \cdot S] + k_{-2}[M\cdot R\cdot S] \\
&\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; + k_3 [M\cdot R \cdot S] - k_4 [Mp][R\cdot S] + k_{-4}[Mp\cdot R\cdot S] + k_5[Mp\cdot R \cdot S] \\[1em]
&\frac{\mathrm{d}[M]}{\mathrm{d}t} = \beta_M - \gamma [M] - k_2 [M][R\cdot S] + k_{-2} [M\cdot R\cdot S] \\[1em]
&\frac{\mathrm{d}[M\cdot R\cdot S]}{\mathrm{d}t} = - \gamma [M\cdot R \cdot S] + k_2 [M][R\cdot S] - k_{-2}[M\cdot R\cdot S] - k_3 [M\cdot R\cdot S] \\[1em]
&\frac{\mathrm{d}[Mp]}{\mathrm{d}t} = -\gamma [Mp] + k_3 [M\cdot R\cdot S] -k_4 [Mp][R\cdot S] + k_{-4}[Mp\cdot R \cdot S] \\[1em]
&\frac{\mathrm{d}[Mp\cdot R\cdot S]}{\mathrm{d}t} = - \gamma [Mp\cdot R \cdot S] + k_4 [Mp][R\cdot S] - k_{-4}[Mp\cdot R\cdot S] - k_5 [Mp\cdot R \cdot S]\\[1em]
&\frac{\mathrm{d}[Mpp]}{\mathrm{d}t} = -\gamma [Mpp] + k_5 [Mp\cdot R \cdot S] - k_6 [E][Mpp] + k_{-6}[E\cdot Mpp] \\
& \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; + k_7 [E\cdot Mpp] - k_8[Ep][Mpp] + k_{-8}[Ep\cdot Mpp] + k_9 [Ep\cdot Mpp]\\[1em]
&\frac{\mathrm{d}[E]}{\mathrm{d}t} = \beta_E - \gamma [E] - k_6 [E][Mpp] + k_{-6}[E\cdot Mpp] \\[1em]
&\frac{\mathrm{d}[E\cdot Mpp]}{\mathrm{d}t} = -\gamma [E\cdot Mpp] + k_6 [E][Mpp] - k_{-6}[E\cdot Mpp] - k_7 [E\cdot Mpp] \\[1em]
&\frac{\mathrm{d}[Ep]}{\mathrm{d}t} = -\gamma [Ep] + k_7[E\cdot Mpp] - k_8[Ep][Mpp] + k_{-8}[Ep\cdot Mpp] \\[1em]
&\frac{\mathrm{d}[Ep\cdot Mpp]}{\mathrm{d}t} = -\gamma [Ep\cdot Mpp] + k_8[Ep][Mpp] - k_{-8}[Ep\cdot Mpp] - k_9 [Ep\cdot Mpp] \\[1em]
&\frac{\mathrm{d}[Epp]}{\mathrm{d}t} = -\gamma [Epp] + k_9[Ep\cdot Mpp]
\end{align}

Note that within the cell these cascades use scaffold proteins to group together kinases at different levels of regulation, which has major impacts on their ability to relay signals. A more complete model is necessary to fully account for these and other additional features of the pathway.

By changing the values of $\beta_R,\beta_M,\beta_E$ for a given $\gamma$, the authors were able to tune the values of the total concentrations of Raf, MEK, and ERK respectively. The authors then varied the MEK and ERK concentrations in their model while keeping the other parameters fixed, and measured the ultrasensitivity (Hill coefficient), threshold value (EC50), and signal strength of the final response. As you can see in the heatmaps below, the authors found that this core MAPK cascade circuit can exhibit a variety of response profiles (marked as symbols on the heatmaps) depending on the concentrations of the downstream kinases.

<div style="width: 750px; margin: auto;">
    
![MAP model heatmaps](figs/MAPK_model_heatmaps.png)

</div>

*Image modified from Fig 5 of O'Shaughnessy et al 2011, Cell*

<!--
JB: 
We need to generate our own figure for the heat maps above.
-->

To visualize the response of the concentration of doubly phosphorylated ERK to total input substrate (S) concentration. This can be calculated by integrating the above dynamical equations for various substrate concentrations and evaluating the steady states. We do this for total concentration of MEK1 and ERK that are indicated by the circle and square in the left heatmap, above. We use the same parameters as in the O'Shaughnessy et al. paper.

<!-- 
Previous text:

Edited out by JB.

Let's see what actual response curves for the model look like at these (and other) points. To do this, we'll need to code up the ODE system above. While in previous chapters we simulated systems of ODEs to determine the evolution of the system over time from its initial condition, in this case we are interested in determining the shape of the *steady-state response profile* of $[Epp]$ to $[S]$. We will therefore need to write another function that will act as a wrapper around the numerical integration function we are already familiar with.

Now let's run this function using parameters from the paper, specifically simulating the system at concentrations of $M_{tot}$ and $E_{tot}$ that were indicated by the circle and square on the paper's heatmap. 
-->

In [5]:
# Write a function to solve the ODEs for a given value of S
def MAPK_rhs(
    x,
    t,
    betaR,
    betaM,
    betaE,
    gamma,
    k1,
    k1m,
    k2,
    k2m,
    k3,
    k4,
    k4m,
    k5,
    k6,
    k6m,
    k7,
    k8,
    k8m,
    k9,
):
    R, RS, M, MRS, Mp, MpRS, Mpp, E, EMpp, Ep, EpMpp, Epp, S = x

    dR_dt = betaR - gamma * R - k1 * R * S - k1m * RS
    dRS_dt = (
        -gamma * RS
        + k1 * R * S
        - k1m * RS
        - k2 * M * RS
        + k2m * MRS
        + k3 * MRS
        - k4 * Mp * RS
        + k4m * MpRS
        + k5 * MpRS
    )
    dM_dt = betaM - gamma * M - k2 * M * RS + k2m * MRS
    dMRS_dt = -gamma * MRS + k2 * M * RS - k2m * MRS - k3 * MRS
    dMp_dt = -gamma * Mp + k3 * MRS - k4 * Mp * RS + k4m * MpRS
    dMpRS_dt = -gamma * MpRS + k4 * Mp * RS - k4m * MpRS - k5 * MpRS
    dMpp_dt = (
        -gamma * Mpp
        + k5 * MpRS
        - k6 * E * Mpp
        + k6m * EMpp
        + k7 * Mpp
        - k8 * Ep * Mpp
        + k8m * EpMpp
        + k9 * Mpp
    )
    dE_dt = betaE - gamma * E - k6 * E * Mpp + k6m * EMpp
    dEMpp_dt = -gamma * EMpp + k6 * E * Mpp - k6m * EMpp - k7 * EMpp
    dEp_dt = -gamma * Ep + k7 * EMpp - k8 * Ep * Mpp + k8m * EpMpp
    dEpMpp_dt = -gamma * EpMpp + k8 * Ep * Mpp - k8m * EpMpp - k9 * EpMpp
    dEpp_dt = -gamma * Epp + k9 * EpMpp
    dS_dt = -k1 * R * S + k1m * RS

    dx_dt = np.array(
        [
            dR_dt,
            dRS_dt,
            dM_dt,
            dMRS_dt,
            dMp_dt,
            dMpRS_dt,
            dMpp_dt,
            dE_dt,
            dEMpp_dt,
            dEp_dt,
            dEpMpp_dt,
            dEpp_dt,
            dS_dt,
        ]
    )
    return dx_dt


# Write a function that solves the ODEs for many values of S and returns the steady state value of Epp
def get_Epp_S_response_curve(
    S_vals,
    betaR,
    betaM,
    betaE,
    gamma,
    k1,
    k1m,
    k2,
    k2m,
    k3,
    k4,
    k4m,
    k5,
    k6,
    k6m,
    k7,
    k8,
    k8m,
    k9,
):
    # Number of time points we want for the solutions
    n = 10000

    # Time points we want for the solution
    t = np.linspace(0, 1e6, n)

    # Initial condition (all in uM)
    R0 = 0.010
    M0 = 0.010
    E0 = 0.010
    x0 = np.zeros(13)
    x0[0] = R0
    x0[2] = M0
    x0[7] = E0

    # Initialize vector to store Epp values
    Epp_vals = np.empty(len(S_vals))

    # Iterate over values of S
    for i, S in enumerate(S_vals):
        x0[12] = S

        # Package parameters into a tuple
        args = (
            betaR,
            betaM,
            betaE,
            gamma,
            k1,
            k1m,
            k2,
            k2m,
            k3,
            k4,
            k4m,
            k5,
            k6,
            k6m,
            k7,
            k8,
            k8m,
            k9,
        )

        # Integrate ODES
        x = scipy.integrate.odeint(MAPK_rhs, x0, t, args=args)

        # Extract Epp value and store it
        Epp_vals[i] = np.median(x[-10:, 11])

    return Epp_vals

# Parameters (units are uM and sec)
gamma = 0.001
k1 = 0.9
k1m = 0.5
k2 = 5.0
k2m = 0.5
k3 = 0.1
k4 = 5.0
k4m = 0.5
k5 = 0.1
k6 = 15.0
k6m = 0.5
k7 = 0.1
k8 = 15.0
k8m = 0.5
k9 = 0.1

# Specific concentrations for the Circle point
Rss_circ = 0.010
Mss_circ = 0.800
Ess_circ = 1.000
betaR_circ = Rss_circ * gamma
betaM_circ = Mss_circ * gamma
betaE_circ = Ess_circ * gamma

# Specific concentrations for the Square point
Rss_sq = 0.010
Mss_sq = 0.020
Ess_sq = 10.000
betaR_sq = Rss_sq * gamma
betaM_sq = Mss_sq * gamma
betaE_sq = Ess_sq * gamma


# Initialize range of S values in logspace
S_vals = np.logspace(-4, 0, 200)

# Solve
Epp_vals_circ = get_Epp_S_response_curve(
    S_vals,
    betaR_circ,
    betaM_circ,
    betaE_circ,
    gamma,
    k1,
    k1m,
    k2,
    k2m,
    k3,
    k4,
    k4m,
    k5,
    k6,
    k6m,
    k7,
    k8,
    k8m,
    k9,
)
Epp_vals_sq = get_Epp_S_response_curve(
    S_vals,
    betaR_sq,
    betaM_sq,
    betaE_sq,
    gamma,
    k1,
    k1m,
    k2,
    k2m,
    k3,
    k4,
    k4m,
    k5,
    k6,
    k6m,
    k7,
    k8,
    k8m,
    k9,
)

# Normalize outputs
Epp_vals_circ /= np.max(Epp_vals_circ)
Epp_vals_sq /= np.max(Epp_vals_sq)

# Build plot
p = bokeh.plotting.figure(
    frame_height=225,
    frame_width=300,
    x_axis_label="S (log uM)",
    y_axis_label="Epp (uM) (Normalized)",
    title=f"MAP kinase cascade, fixed parameters",
    x_axis_type='log',
    x_range=[1e-4, 1],
)

# Plot outputs
p.line(
    S_vals,
    Epp_vals_circ,
    line_width=3,
    color="#1f77b4",
    legend_label="◯ (switchlike)",
)
p.line(S_vals, Epp_vals_sq, line_width=3, color="orange", legend_label="▢ (graded)")

p.legend.location = "top_left"

# NO NEED FOR THIS
# Import in heatmap
# from bokeh.plotting import figure, show, output_file
# from bokeh.models import Div


# hm = bokeh.plotting.figure(frame_height=375, frame_width=375,)
# hm.image_url(url=["figs/MAPK_n_heatmap.png"], x=0, y=1, w=0.8, h=0.6)
# hm.xaxis.visible = False
# hm.yaxis.visible = False
# hm.xgrid.grid_line_color = None
# hm.ygrid.grid_line_color = None
# hm.outline_line_alpha = 0

# # Build layout
# MAPK_layout = bokeh.layouts.row(hm, p)

# bokeh.io.show(MAPK_layout)

bokeh.io.show(p)

We can indeed see that the minimal MAP kinase cascade is able to exhibit both graded and switchlike responses *for the same choice of kinetic parameters*. The only difference between the two conditions above are the values total concentrations of MEK1 and ERK2, which are tuned by changing the values of $\beta_M$ and $\beta_E$ with $\gamma$ fixed. We can now see conclusively that the kinase cascade architecture enables cells to dynamically change the qualitative nature of its response, via its steepness, to the same environmental stimulus simply by changing the concentration of the kinases in the cascade.

## Conclusion: phosphorylation cascades provide robust, tunable protein-level amplification

Phosphorylation appears to be a powerful mechanism for generating protein amplification systems. We have seen that these systems can provide remarkable capabilities. In a two-component system—the simplest phosphorylation cascade one can imagine—bifunctional kinases can generate linear input-output relationships, whose gain can be tuned by a single rate constant. This ability allows a pathway to convert a signal from one form to another, accurately preserving information about its level. Amazingly, this entire input-output system can be robust to the levels of its own components. Perhaps it is at least in part the combination of simplicity of design and the general usefulness of this function that accounts for the tremendous proliferation of two component systems among bacteria. E. coli has ~29 of them. B. subtilis has at least 30. And other species have even more. 

Then we examined MAP kinase cascades, one of the best studied and most central circuits within the cell. We found that these systems can generate different levels of amplification and, critically, can allow the cell to modulate the sensitivity, from linear to ultrasensitive, just by modulating the concentrations of its own components. In other words, the cascade functions as a tunable amplifier, whose gain and sensitivity can differ depending on the cellular context. On the one hand, this is a powerful feature for the cell that enhances the flexibility of one of its core pathways. On the other hand, it means that just knowing that a MAP kinase cascade is involved in a process is not enough to predict how it will amplify signals. To know that, we also need to know the expression levels of kinases, and likely other related components as well. 

In the future, one can imagine having a predictive model that would tell us what the gain and sensitivity of a circuit will look like based on the expression levels of its protein components—something that is increasingly available thanks to high-throughput "omics" methods. One can also consider other types of protein amplifiers. For example, within the programmed cell death pathways there are cascades of proteases that activate each other. How do such protease cascades differ in their amplification abilities compared to phosphorylation cascades?

## Computing environment

In [6]:
%load_ext watermark
%watermark -v -p numpy,scipy,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.2.0

numpy     : 1.21.2
scipy     : 1.7.3
bokeh     : 2.4.2
jupyterlab: 3.3.2

